In [1]:
### Find Missing Utilities 
# These are utilities that aren't in EIA923 2015 Generation & Fuel table, which
# was used to make the initial list of utilities

In [2]:
import os, sys
sys.path.append(os.path.abspath(os.path.join('..','..')))
from pudl import ferc1, pudl, models, models_ferc1, settings, constants, eia923, models_eia923
from pudl.eia923 import get_eia923_page
from pudl.eia923 import get_eia923_xlsx, get_eia923_column_map
from pudl.eia923 import get_eia923_plant_info, get_eia923_operator_info
from pudl.pudl import *
from pudl.constants import tab_map_eia923
import numpy as np
import pandas as pd
from pandas import DataFrame
import csv
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.sql import select
from sqlalchemy import Integer, String, Numeric, Boolean, Float

In [3]:
%load_ext autoreload
%autoreload 1
%aimport pudl.pudl
%aimport pudl.ferc1
%aimport pudl.eia923
%aimport pudl.constants
%aimport pudl.settings
%aimport pudl.models
%aimport pudl.models_eia923

In [ ]:
eia923_years=[2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]

In [ ]:
eia923_xlsx = get_eia923_xlsx(eia923_years)

Reading EIA 923 spreadsheet data for 2009.
Reading EIA 923 spreadsheet data for 2010.
Reading EIA 923 spreadsheet data for 2011.
Reading EIA 923 spreadsheet data for 2012.
Reading EIA 923 spreadsheet data for 2013.
Reading EIA 923 spreadsheet data for 2014.
Reading EIA 923 spreadsheet data for 2015.


In [ ]:
eia923_plants = get_eia923_operator_info(eia923_years, eia923_xlsx)
eia923_operator_ids = eia923_plants[['operator_id']]

In [ ]:
eia923_plants = eia923_plants.dropna(subset=['operator_id'])
eia923_plants.head(2)

In [ ]:
output_tab_plants = pd.read_excel('../id_mapping/mapping_eia923_ferc1.xlsx', 'utilities_output', na_values='', keep_default_na=False,
                              converters={'plant_id': int,
                                          'plant_name': str,
                                          'respondent_id_ferc1': int,
                                          'respondent_name_ferc1': str,
                                          'plant_name_ferc1': str,
                                          'plant_id_eia923': int,
                                          'plant_name_eia923': str,
                                          'operator_name_eia923': str,
                                          'operator_id_eia923': int})
output_operator_ids = output_tab_plants[['operator_id_eia923']]
output_operator_ids = output_operator_ids.dropna()

In [ ]:
output_operator_ids = output_operator_ids.rename(columns={'operator_id_eia923': 'operator_id'})

In [ ]:
merged = eia923_operator_ids.merge(output_operator_ids, indicator=True, how='outer')
merged = merged[merged['_merge'] == 'right_only']
merged = merged.drop_duplicates()
del merged['_merge']

In [ ]:
merged.shape

In [ ]:
missing_operators = merged.rename(columns={0: 'operator_id',})
missing_operators.drop_duplicates()
eia923_plants.drop_duplicates('operator_id')
missing_operators["operator_id"] = missing_operators.operator_id.astype(int)
eia923_plants["operator_id"] = eia923_plants.operator_id.astype(int)

In [ ]:
missing_operators = pd.merge(missing_operators, eia923_plants, on=['operator_id'], how='left')

In [ ]:
output_tab_plants = pd.read_excel('../id_mapping/mapping_eia923_ferc1.xlsx', 'plants_output', na_values='', keep_default_na=False,
                              converters={'plant_id': int,
                                          'plant_name': str,
                                          'respondent_id_ferc1': int,
                                          'respondent_name_ferc1': str,
                                          'plant_name_ferc1': str,
                                          'plant_id_eia923': int,
                                          'plant_name_eia923': str,
                                          'operator_name_eia923': str,
                                          'operator_id_eia923': int})
output_operator_ids = output_tab_plants[['operator_id_eia923', 'operator_name_eia923']]
output_operator_ids = output_operator_ids.dropna()

In [ ]:
output_operator_ids = output_operator_ids.rename(columns={'operator_id_eia923': 'operator_id'})

In [ ]:
missing_operators = pd.merge(missing_operators, output_operator_ids, on=['operator_id'], how='left')

In [ ]:
missing_operators = missing_operators.drop_duplicates('operator_id')
del missing_operators['operator_name']
missing_operators = missing_operators.rename(columns={'operator_name_eia923': 'operator_name'})

In [ ]:
missing_operators.head(20)

In [ ]:
missing_operators.to_csv('../id_mapping/missing_operators.csv', index=False)